In [1]:
import numpy as np
import os

#Loading observation time and date data

data = np.loadtxt('IRIS_Flare_Observation_List:', usecols=(0,1,2), skiprows=11, dtype='str', delimiter=' ', encoding = 'utf-8')

realdate = data[:, 0]
realtime = data[:, 1]
realmag = data[:,2]

times = []
time = []
mag = []
for i in range(len(realtime)):
    try:
        time.append(realtime[i][0:2] + realtime[i][3:5])
        times.append(realdate[i])
        mag.append(realmag[i][0:4])
    except ValueError:
        continue
    

In [3]:
#Finding Files of the observations and filtering for single-slit

import irisreader as ir
from irisreader import observation
ir.config.verbosity_level=0

def list_of_files(directory_path):
    try:
        #Get the list of files in the directory
        entries = os.listdir(directory_path)
        file_names = []
        for i in range(len(entries)):
            file_names.append(entries[i])
        #Filter out only files (not directories)
        return file_names
    except FileNotFoundError:
        return f"Error: The directory '{directory_path}' does not exist."
    except PermissionError:
        return f"Error: Permission denied to access '{directory_path}'."
    except Exception as e:
        return f"An unexpected error occurred: {e}"

def Data_Filter(path, mag2): # Only include datasets with 1 raster step and have a observable Mg II subordinate line
    newpath = [] 
    mag3 = []
    for n in range(len(path)):
        path0 = "/sml/iris/" + str(path[n][0:4] + "/" + str(path[n][4:6]) + "/" + str(path[n][6:8]))
        try:
            os.chdir(path0)
            try:
                obs = observation(path[n], keep_null=True)
                raster = obs.raster("Mg II k").get_global_raster_step(0, 1)
                if raster == 1:
                    if obs.raster("Mg II k").headers[0]["WAVEMIN"] + 0.00001 <= 2791.2:
                        newpath.append(path[n])
                        mag3.append(mag2[n])
            except ValueError:
                print(f"Error: This data '{path[n]}' contains no data")
            except OSError as e:
                print(f"OSError: {e} occurred while processing '{path[n]}'")
            except UnicodeDecodeError:
                print(f"UnicodeDecodeError: Cannot decode '{path[n]}'")
        except FileNotFoundError:
            print(f"Error: The directory '{path0}' does not exist.")
        except OSError as e:
            print(f"OSError: {e} occurred while changing directory to '{path0}'")
        except UnicodeDecodeError:
            print(f"UnicodeDecodeError: Cannot decode '{path0}'")

    return newpath, mag3


path = []
mag1 = []
i=0
for n in range(len(times)):
    try: 
        #Directory path generation
        path0 = "/sml/iris/" + str(times[n][0:4] + "/" + str(times[n][4:6]) + "/" + str(times[n][6:8]))
        os.chdir( path0)
        directory_path = path0  
        files_in = list_of_files(directory_path)
        files = sorted(files_in, key=lambda f: int(f[9:13]))
        found = False
        #Sort for only files that could have genrated the flare (i.e. the observed flare time must be larger than the 'correct' observation's time and smaller than the proceeding observation time)
        for j in range(len(files)-1):
            if files[j][0:8] == times[n] and files[j][9:13] <= str(time[n][0:4]) and files[j+1][9:13] >= str(time[n][0:4]):
                path.append(files[j])
                mag1.append(mag[n])
                found = True
        if found == False:
            path.append(files[-1])
            mag1.append(mag[n])
    except FileNotFoundError:
        continue

#Filtering for only flares with useful data for the thesis
Usedata, mag2 = Data_Filter(path, mag1)

UsableData = []
Finalmag = []
#A precaution to eliminate doubles in case two flares happened during the same observation
for i in range(len(Usedata)):
    Double = False
    for j in range(len(Usedata)):
        if Usedata[i] == Usedata[j] and i != j:
            Double = True
    if Double == False:
        UsableData.append(Usedata[i])
        Finalmag.append(mag2[i])



OSError: Header missing END card. occurred while processing '20150314_173912_3860607059'
OSError: Header missing END card. occurred while processing '20150314_173912_3860607059'
OSError: Header missing END card. occurred while processing '20150314_173912_3860607059'
OSError: Header missing END card. occurred while processing '20150314_173912_3860607059'
OSError: Header missing END card. occurred while processing '20150314_173912_3860607059'
OSError: Header missing END card. occurred while processing '20150314_173912_3860607059'


In [4]:
for i in range(len(UsableData)):
    if Finalmag[i][0] == 'X':
    #if Finalmag[i][0] == 'M':
    #if Finalmag[i][0] == 'C':
        print(UsableData[i], i)

20140910_112825_3860259453 9
20141024_205222_3860111353 11
20220330_161411_3660259102 63
